In [1]:
! pip install transformers
! pip install keras_tuner

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [7]:
!echo $CUDA_VISIBLE_DEVICES

0


In [8]:
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Bidirectional, Dropout, GlobalMaxPool1D, concatenate, BatchNormalization
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import GridSearch
from keras_tuner import HyperModel, Objective
from keras import regularizers
from decimal import Decimal
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print("GPUs available: ", gpus)

GPUs available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
if gpus:
    # Assuming you want to use the first GPU if available
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("Using GPU: ", gpus[0])
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

Using GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [11]:
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Memory growth enabled


In [12]:
import keras
SEED = 42
tf.random.set_seed(SEED)

DATASET PROCESSATO

In [13]:
choice = 'umberto' #umberto

In [14]:
max_length = 0
if choice == 'alberto':
    training_data = pd.read_csv('archive/alberto_preprocessing_training_textual_haspeede2.csv')
    test_data = pd.read_csv('archive/alberto_preprocessing/alberto_preprocessing_test_textual.csv')
    max_length = 290
    print('alberto chosen')
else:
    training_data = pd.read_csv('archive/umberto_preprocessing_training_textual_haspeede2.csv')
    test_data = pd.read_csv('archive/umberto_preprocessing/umberto_preprocessing_test_textual.csv')
    max_length = 357
    print('umberto chosen')

print(max_length)

umberto chosen
357


In [15]:
training_data

,anonymized_text,label
0,[user] con tutte le denunce che si sta beccand...,1
1,[hashtag] prescrizione [/hashtag] i t re magi ...,0
2,il [hashtag] movimento cinque stelle [/hashtag...,1
3,la [hashtag] lega [/hashtag] e il [hashtag] mo...,0
4,che cosa cambia questa [hashtag] legge [/hasht...,0
...,...,...
12432,gli stati nazionali devono essere pronti a rin...,0
12433,il ministro dell interno della germania [hasht...,0
12434,[hashtag] salvini [/hashtag] in italia troppi ...,0
12435,[user] [user] chi giubila in buona fede non ha...,0


In [16]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12437 entries, 0 to 12436
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   anonymized_text  12437 non-null  object
 1   label            12437 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 194.5+ KB


In [17]:
test_data

,anonymized_tweet_id,anonymized_text,label,dataset
0,424801448454884,questo anno [hashtag] babbo natale [/hashtag] ...,1,test_politics
1,775194088981616,adesso che in mezzo alla strada grazie al verg...,1,test_politics
2,386928936347190,circondatevi di persone che non sono diventate...,0,test_politics
3,957546674360002,seriamente per capire se un ladro mi entra in ...,0,test_politics
4,530526299555950,che poi è probabile che [hashtag] spataro [/ha...,0,test_politics
...,...,...,...,...
4395,391599381944152,[user] esatto brava ti dico di piu i nazisti n...,0,test_religious
4396,974317501584882,[user] togliere la parola non vaccinati e mett...,0,test_religious
4397,739414582677163,[user] [user] lo sai o no che i musulmani rico...,0,test_religious
4398,470933865054049,berlino memoriale per gli ebrei assassinati d ...,0,test_religious


In [18]:
test_data['dataset'].unique()

array(['test_politics', 'test_religious'], dtype=object)

### only for Task B

In [19]:
test_religious_data = test_data.loc[test_data['dataset'] == 'test_religious']
test_politics_data = test_data.loc[test_data['dataset'] == 'test_politics']

In [20]:
test_religious_data

,anonymized_tweet_id,anonymized_text,label,dataset
605,379748472796095,voglio abituare tutti gli abitanti del luogo e...,0,test_religious
606,195027074607893,l europa laica e comunista esce allo scoperto ...,0,test_religious
607,97213378242080,[user] gli ebrei insieme ai liberali e altre c...,0,test_religious
608,894894959501945,[user] la mussulmana lombarda per portarla a c...,1,test_religious
609,439618972593653,denunciare coloro che vogliono limitare le lib...,0,test_religious
...,...,...,...,...
4395,391599381944152,[user] esatto brava ti dico di piu i nazisti n...,0,test_religious
4396,974317501584882,[user] togliere la parola non vaccinati e mett...,0,test_religious
4397,739414582677163,[user] [user] lo sai o no che i musulmani rico...,0,test_religious
4398,470933865054049,berlino memoriale per gli ebrei assassinati d ...,0,test_religious


In [21]:
"""# Function to count words in a sentence
def count_words(sentence):
    words = sentence.split()
    return len(words)
sentences = training_data['anonymized_text'].copy()
# Apply the function to each sentence in the series
word_counts = sentences.apply(count_words)

# Find the maximum word count in the series
max_word_count = word_counts.max()

max_word_count #in the test was 106 the max."""

"# Function to count words in a sentence\ndef count_words(sentence):\n    words = sentence.split()\n    return len(words)\nsentences = training_data['anonymized_text'].copy()\n# Apply the function to each sentence in the series\nword_counts = sentences.apply(count_words)\n\n# Find the maximum word count in the series\nmax_word_count = word_counts.max()\n\nmax_word_count #in the test was 106 the max."

In [22]:
# Load model directly
from transformers import AutoTokenizer

#max_length = 300

tokenizer = AutoTokenizer.from_pretrained("idb-ita/gilberto-uncased-from-camembert")

def tokenize_function(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=max_length,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


## Train-Test Split + Tweets transformation for the model

In [23]:
from sklearn.model_selection import train_test_split

labels = training_data['label']

X_train, X_val, y_train, y_val = train_test_split(training_data, labels, test_size=0.20, random_state=SEED, stratify=labels)

In [24]:
X_train.head()

,anonymized_text,label
1777,[hashtag] decreto cura italia [/hashtag] [hash...,0
216,il premier [hashtag] conte [/hashtag] ha appen...,0
4524,[hashtag] salvini pagliaccio [/hashtag] cosa n...,1
5452,il [hashtag] movimento cinque stelle [/hashtag...,0
5742,vaticano contro trump appello alla chiesa usa ...,0


In [25]:
def dumb_fun(Xmask):
    num_ones = np.count_nonzero(Xmask[i,:] == 1)
    if num_ones >= 260:
        print(Xmask[i,:])
        print(f"The number of ones in the ndarray is: {num_ones}")

### **Normalize dataset**

In [26]:
"""# Initialize the scaler
scaler = StandardScaler()
columns_to_normalize = ['retweet_count', 'favorite_count', 'statuses_count', 'followers_count', 'friends_count']

# Fit the scaler on the training data and transform it
X_train[columns_to_normalize] = scaler.fit_transform(X_train[columns_to_normalize])

# Transform the validation and test data with the same scaler
X_val[columns_to_normalize] = scaler.transform(X_val[columns_to_normalize])
test_data[columns_to_normalize] = scaler.transform(test_data[columns_to_normalize])"""

"# Initialize the scaler\nscaler = StandardScaler()\ncolumns_to_normalize = ['retweet_count', 'favorite_count', 'statuses_count', 'followers_count', 'friends_count']\n\n# Fit the scaler on the training data and transform it\nX_train[columns_to_normalize] = scaler.fit_transform(X_train[columns_to_normalize])\n\n# Transform the validation and test data with the same scaler\nX_val[columns_to_normalize] = scaler.transform(X_val[columns_to_normalize])\ntest_data[columns_to_normalize] = scaler.transform(test_data[columns_to_normalize])"

In [27]:
X_train.head()

,anonymized_text,label
1777,[hashtag] decreto cura italia [/hashtag] [hash...,0
216,il premier [hashtag] conte [/hashtag] ha appen...,0
4524,[hashtag] salvini pagliaccio [/hashtag] cosa n...,1
5452,il [hashtag] movimento cinque stelle [/hashtag...,0
5742,vaticano contro trump appello alla chiesa usa ...,0


In [28]:
X_val.head()

,anonymized_text,label
5481,[user] [user] potreste segnalare a [hashtag] d...,0
2900,gli [hashtag] animali [/hashtag] [hashtag] dom...,0
5296,[user] è un vizio dei mattei [hashtag] renzi [...,1
6843,eh si anche lui niente obiettivo magistrati tu...,0
10874,[hashtag] roma [/hashtag] [number] [number] en...,1


In [29]:
test_data.head()

,anonymized_tweet_id,anonymized_text,label,dataset
0,424801448454884,questo anno [hashtag] babbo natale [/hashtag] ...,1,test_politics
1,775194088981616,adesso che in mezzo alla strada grazie al verg...,1,test_politics
2,386928936347190,circondatevi di persone che non sono diventate...,0,test_politics
3,957546674360002,seriamente per capire se un ladro mi entra in ...,0,test_politics
4,530526299555950,che poi è probabile che [hashtag] spataro [/ha...,0,test_politics


In [30]:
# initialize two arrays for input tensors
Xids = np.zeros((len(X_train['anonymized_text']), max_length))
Xmask = np.zeros((len(X_train['anonymized_text']), max_length))

for i, sentence in enumerate(X_train['anonymized_text']):
    Xids[i, :], Xmask[i, :] = tokenize_function(sentence)
labels = y_train
BATCH_SIZE = 128  # we will use batches of 128

# load arrays into tensorflow dataset
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

print(dataset)

# create a mapping function that we use to restructure our dataset
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# using map method to apply map_func to dataset
dataset = dataset.map(map_func)

# batch data
dataset = dataset.batch(BATCH_SIZE)
train = dataset
del dataset

<TensorSliceDataset shapes: ((357,), (357,), ()), types: (tf.float64, tf.float64, tf.int64)>


In [31]:
print(tokenizer.convert_ids_to_tokens(Xids[0,:]))

['▁s', '▁[', 'ha', 'shtag', ']', '▁decreto', '▁cura', '▁italia', '▁[', '/', 'ha', 'shtag', ']', '▁[', 'ha', 'shtag', ']', '▁decreto', '▁marzo', '▁[', '/', 'ha', 'shtag', ']', '▁usare', '▁la', '▁[', 'ha', 'shtag', ']', '▁cd', 'p', '▁[', '/', 'ha', 'shtag', ']', '▁per', '▁fronteggiare', '▁la', '▁crisi', '▁da', '▁[', 'ha', 'shtag', ']', '▁co', 'vi', 'd', '▁[', 'nu', 'mber', ']', '▁[', '/', 'ha', 'shtag', ']', '▁non', '▁si', '▁può', '▁para', 'gon', 'arla', '▁alla', '▁[', 'ha', 'shtag', ']', '▁k', 'f', 'w', 'bank', '▁[', '/', 'ha', 'shtag', ']', '▁quest', '▁ultima', '▁è', '▁infalli', 'bile', '▁la', '▁[', 'ha', 'shtag', ']', '▁cd', 'p', '▁[', '/', 'ha', 'shtag', ']', '▁viene', '▁garantita', '▁con', '▁i', '▁suoi', '▁depositi', '▁gran', '▁parte', '▁risparmi', '▁degli', '▁italiani', '▁e', '▁solo', '▁in', '▁parte', '▁dallo', '▁stato', '▁[', 'user', ']', 'on', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

In [32]:
"""tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

phrase = " <hashtag> movimento cinque stelle </hashtag>"
phrase1 = " <number> salvini a casa"
    #X_train[X_train['anonymized_text'].str.contains(r'.*[0-9].*', regex=True)]['anonymized_text'][5186]

Xids[i, :], Xmask[i, :] = tokenize_function(phrase, phrase1)

print(tokenizer.convert_ids_to_tokens(Xids[0,:]))"""

'tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")\n\nphrase = " <hashtag> movimento cinque stelle </hashtag>"\nphrase1 = " <number> salvini a casa"\n    #X_train[X_train[\'anonymized_text\'].str.contains(r\'.*[0-9].*\', regex=True)][\'anonymized_text\'][5186]\n\nXids[i, :], Xmask[i, :] = tokenize_function(phrase, phrase1)\n\nprint(tokenizer.convert_ids_to_tokens(Xids[0,:]))'

In [33]:
"""tokenizer = AutoTokenizer.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")

phrase = "[hashtag] [/hashtag] [url] [email] [percent] [money] [phone] [user] [time] [url] [date] [number] [allcaps] ciao! [/allcaps] m[number]s"

Xids[i, :], Xmask[i, :] = tokenize_function(phrase)

print(tokenizer.convert_ids_to_tokens(Xids[i,:]))"""

'tokenizer = AutoTokenizer.from_pretrained("Musixmatch/umberto-commoncrawl-cased-v1")\n\nphrase = "[hashtag] [/hashtag] [url] [email] [percent] [money] [phone] [user] [time] [url] [date] [number] [allcaps] ciao! [/allcaps] m[number]s"\n\nXids[i, :], Xmask[i, :] = tokenize_function(phrase)\n\nprint(tokenizer.convert_ids_to_tokens(Xids[i,:]))'

In [34]:
XidsVal = np.zeros((len(X_val['anonymized_text']), max_length))
XmaskVal = np.zeros((len(X_val['anonymized_text']), max_length))

for i, sentence in enumerate(X_val['anonymized_text']):
    XidsVal[i, :], XmaskVal[i, :] = tokenize_function(sentence)

labels_val = y_val

dataset = tf.data.Dataset.from_tensor_slices((XidsVal, XmaskVal, labels_val))
dataset = dataset.map(map_func)
dataset = dataset.batch(BATCH_SIZE)
validation = dataset
del dataset

In [35]:
XidsTestPolitics = np.zeros((len(test_politics_data['anonymized_text']), max_length))
XmaskTestPolitics = np.zeros((len(test_politics_data['anonymized_text']), max_length))

for i, sentence in enumerate(test_politics_data['anonymized_text']):
    XidsTestPolitics[i, :], XmaskTestPolitics[i, :] = tokenize_function(sentence)

labelsTPolitics = test_politics_data['label']

dataset = tf.data.Dataset.from_tensor_slices((XidsTestPolitics, XmaskTestPolitics, labelsTPolitics))
dataset = dataset.map(map_func)
dataset = dataset.batch(BATCH_SIZE)
test_politics = dataset

del dataset  # delete dataset to free up disk-space

In [36]:
XidsTestReligious = np.zeros((len(test_religious_data['anonymized_text']), max_length))
XmaskTestReligious = np.zeros((len(test_religious_data['anonymized_text']), max_length))

for i, sentence in enumerate(test_religious_data['anonymized_text']):
    XidsTestReligious[i, :], XmaskTestReligious[i, :] = tokenize_function(sentence)

labelsTReligious = test_religious_data['label']

dataset = tf.data.Dataset.from_tensor_slices((XidsTestReligious, XmaskTestReligious, labelsTReligious))
dataset = dataset.map(map_func)
dataset = dataset.batch(BATCH_SIZE)
test_religious = dataset

del dataset  # delete dataset to free up disk-space

## Model Training/Validation/Testing

In [37]:
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))
    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [39]:
from transformers import TFAutoModel
embed_dim = 768  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

class MyHyperModel(HyperModel):
    def build(self, hp):
        model = TFAutoModel.from_pretrained("idb-ita/gilberto-uncased-from-camembert", from_pt=True) #m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 #Musixmatch/umberto-commoncrawl-cased-v1

        input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
        mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
        
        # we consume the last_hidden_state tensor from bert (discarding pooled_outputs)
        embeddings = model(input_ids, attention_mask=mask)[0]

        #X = TransformerBlock(embed_dim=embed_dim,num_heads=num_heads, ff_dim=ff_dim)(embeddings)
        X = Bidirectional(LSTM(units=hp.Choice('LSTM_units', values=[64, 128]), recurrent_dropout=hp.Choice('LSTM_recurrent_dropout', values=[0.0,0.3]), return_sequences= True))(embeddings)
        X = GlobalMaxPool1D()(X)
        X = Dropout(rate=hp.Choice('rate_dropout', values=[0.6, 0.7]))(X)
        #X = BatchNormalization()(X)
        X = Dense(units=hp.Choice('dense_units', values=[16,32,64]),kernel_regularizer=regularizers.L2(hp.Choice('reg_value', [0.0,0.01,0.001])), activation="relu")(X)
        y = Dense(1, activation='sigmoid', name='outputs')(X)

        # define input and output layers of our model
        best_model = Model(inputs=[input_ids, mask], outputs=y)

        # freeze the BERT layer - otherwise we will be training 100M+ parameters...
        best_model.layers[2].trainable = False

        best_model.compile(optimizer=Adam(learning_rate = hp.Choice('learning_rate', values=[0.01, 1e-3]), clipnorm= hp.Choice('clip_norm', values= [1,3])), loss="binary_crossentropy", metrics=[F1Score()])
        return best_model

0.001 l.r.

clipnorm 2, dropout 05 val:0.873 test:0.896 | 06 val:0.869 test:0.89 | 07 val:0.871 test: 0.895

clipnorm 5, dropout 05 val:nondegnodinota test:nondegnodinota | 06 val:0.870 test:0.90 | (18 epoche) 07 val:0.8738 test: 0.8986 (molto bilanciate le classi, no overfitting) (LSTM singola)

0.01 l.r.

clipnorm 3, dropout 07 val: 0.88 test: 0.905 LSTM 64

In [40]:
tuner = GridSearch(
    hypermodel = MyHyperModel(),
    objective = Objective('val_loss', 'min'),
    max_trials = 20,
    seed = SEED,
    executions_per_trial = 1,
    overwrite = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFCamembertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [41]:
tuner.search_space_summary(extended=True)

Search space summary
Default search space size: 7
LSTM_units (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128], 'ordered': True}
LSTM_recurrent_dropout (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.3], 'ordered': True}
rate_dropout (Choice)
{'default': 0.6, 'conditions': [], 'values': [0.6, 0.7], 'ordered': True}
dense_units (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
reg_value (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.01, 0.001], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}
clip_norm (Choice)
{'default': 1, 'conditions': [], 'values': [1, 3], 'ordered': True}


In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True, verbose=1)
tuner.search(train, epochs=15, validation_data=validation, callbacks=[es])

Trial 10 Complete [00h 50m 42s]
val_loss: 0.5034623742103577

Best val_loss So Far: 0.46731340885162354
Total elapsed time: 08h 40m 35s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
64                |64                |LSTM_units
0                 |0                 |LSTM_recurrent_dropout
0.6               |0.6               |rate_dropout
16                |16                |dense_units
0.001             |0                 |reg_value
0.001             |0.001             |learning_rate
1                 |3                 |clip_norm



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFCamembertModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFCamembertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFCamembertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and

Epoch 1/15


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


78/78 [==============================] - ETA: 0s - loss: 0.6985 - f1_score: 0.1940

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


78/78 [==============================] - 224s 3s/step - loss: 0.6985 - f1_score: 0.1940 - val_loss: 0.6731 - val_f1_score: 0.1460
Epoch 2/15
78/78 [==============================] - 192s 2s/step - loss: 0.6680 - f1_score: 0.2963 - val_loss: 0.6482 - val_f1_score: 0.3304
Epoch 3/15
78/78 [==============================] - 192s 2s/step - loss: 0.6378 - f1_score: 0.4808 - val_loss: 0.6091 - val_f1_score: 0.5750
Epoch 4/15
78/78 [==============================] - 192s 2s/step - loss: 0.6086 - f1_score: 0.5401 - val_loss: 0.5786 - val_f1_score: 0.6406
Epoch 5/15
78/78 [==============================] - 192s 2s/step - loss: 0.5808 - f1_score: 0.5880 - val_loss: 0.5653 - val_f1_score: 0.6775
Epoch 6/15
78/78 [==============================] - 191s 2s/step - loss: 0.5616 - f1_score: 0.6097 - val_loss: 0.5355 - val_f1_score: 0.6677
Epoch 7/15
78/78 [==============================] - 193s 2s/step - loss: 0.5445 - f1_score: 0.6240 - val_loss: 0.5227 - val_f1_score: 0.6681
Epoch 8/15
78/78 [======

In [ ]:
# Random Search 10 best results
tuner.results_summary()

In [ ]:
# Focus on Best Trial
tuner.oracle.get_best_trials()[0].get_state()

In [ ]:
# Focus on Best Trial
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

In [ ]:
# Best Model
best_model = tuner.hypermodel.build(best_hps)
best_model.summary()

In [ ]:
# Training of best model
es = EarlyStopping(monitor='val_f1_score', patience=10, restore_best_weights=True, verbose=1, mode="max")
history = best_model.fit(train, batch_size=BATCH_SIZE, epochs=60, callbacks=es, validation_data=validation)

In [ ]:
#Accuracy plot
plt.plot(history.history["f1_score"], label='f1_score')
plt.plot(history.history["val_f1_score"], label='val_f1_score')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim()
plt.xticks(np.arange(len(history.history["val_f1_score"])), np.arange(1, len(history.history["val_f1_score"])+1))
plt.legend(loc = 'lower right')

In [ ]:
#Loss plot
plt.plot(history.history["loss"], label='loss')
plt.plot(history.history["val_loss"], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim()
plt.xticks(np.arange(len(history.history["loss"])), np.arange(1, len(history.history["loss"])+1))
plt.xticks(np.arange(len(history.history["val_loss"])), np.arange(1, len(history.history["val_loss"])+1))
plt.legend(loc = 'upper right')

In [ ]:
pred = best_model.predict(test) > 0.5

In [ ]:
pred_politics = best_model.predict(test_politics) > 0.5

In [ ]:
# Getting the Classification Report
print(classification_report(list(labelsTPolitics), pred_politics, digits=4))

In [ ]:
pred_religious = best_model.predict(test_religious) > 0.5

In [ ]:
# Getting the Classification Report
print(classification_report(list(labelsTReligious), pred_religious, digits=4))